In [1]:
# Introduction

In [2]:
## Loading Libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.model_selection import cross_val_score, KFold, StratifiedShuffleSplit, cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
## Loading Dataset

In [12]:
filename='Dataset O.xlsx'
df=pd.read_excel(filename, index_col=0)
df.head()

,1035.33569,1109.43421,1111.48872,1134.0883,1138.04756,1138.19732,1142.15603,1208.05058,1209.94587,1210.10509,3260.50387,3262.12911,3404.31941,3405.92573,Target
0,0.31346,0.209295,0.204285,0.178940,0.18338,0.184495,0.18651,0.130745,0.13137,0.132035,0.129130,0.12930,0.111235,0.11080,0.0
1,0.29539,0.196555,0.192050,0.168305,0.17230,0.173360,0.17541,0.124610,0.12529,0.126085,0.118080,0.11837,0.101750,0.10133,0.0
2,0.32639,0.213745,0.208790,0.180820,0.18490,0.186035,0.18831,0.132060,0.13274,0.133540,0.134940,0.13514,0.115065,0.11472,0.0
3,0.30666,0.204135,0.199250,0.174510,0.17863,0.179700,0.18189,0.127795,0.12844,0.129200,0.126085,0.12632,0.107810,0.10752,0.0
4,0.31240,0.204615,0.199895,0.172660,0.17674,0.177790,0.17987,0.124430,0.12510,0.125890,0.123430,0.12374,0.105745,0.10564,0.0


In [13]:
X, Y=df.iloc[:, :-1], df.iloc[:, -1]

In [14]:
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

for train_index, test_index in stratified_split.split(X, Y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    
print('X_train: {}     Y_train: {} \n X_test: {}     Y_test:{} '.format(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape))

X_train: (143, 14)     Y_train: (143,) 
 X_test: (36, 14)     Y_test:(36,) 


In [15]:
## Model developmenmt

In [16]:
### 1. Linear Regresssion

In [17]:
model1 = LinearRegression()

In [18]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

Y_pred_train = cross_val_predict(model1, X_train, Y_train, cv=cv)

mse_train = -np.mean(cross_val_score(model1, X_train, Y_train, cv=cv, scoring='neg_mean_squared_error'))
rmse_train = np.sqrt(mse_train)
mae_train = np.mean(cross_val_score(model1, X_train, Y_train, cv=cv, scoring='neg_mean_absolute_error'))
r2_train_scores = cross_val_score(model1, X_train, Y_train, cv=cv, scoring='r2')
r2_train_mean = np.mean(r2_train_scores)

residuals_train = Y_train - Y_pred_train
std_dev_res_train = np.std(residuals_train)
mean_y_train = np.mean(Y_train)
rpd_train = std_dev_res_train / mean_y_train

print('\nTraining Set: ')
print(f'Mean Squared Error: {mse_train}')
print(f'Root Mean Squared Error: {rmse_train}')
print(f'Mean Absolute Error: {mae_train}')
print(f'R-squared: {r2_train_mean}')
print(f'Relative Percent Difference: {rpd_train}')

Y_pred_test = cross_val_predict(model1, X_test, Y_test, cv=cv)

mse_test = -np.mean(cross_val_score(model1, X_test, Y_test, cv=cv, scoring='neg_mean_squared_error'))
rmse_test = np.sqrt(mse_test)
mae_test = np.mean(cross_val_score(model1, X_test, Y_test, cv=cv, scoring='neg_mean_absolute_error'))
r2_test_scores = cross_val_score(model1, X_test, Y_test, cv=cv, scoring='r2')
r2_test_mean = np.mean(r2_test_scores)

residuals_test = Y_test - Y_pred_test
std_dev_res_test = np.std(residuals_test)
mean_y_test = np.mean(Y_test)
rpd_test = std_dev_res_test / mean_y_test

print('\nTesting Set: ')
print(f'Mean Squared Error: {mse_test}')
print(f'Root Mean Squared Error: {rmse_test}')
print(f'Mean Absolute Error: {mae_test}')
print(f'R-squared: {r2_test_mean}')
print(f'Relative Percent Difference: {rpd_test}')


Training Set: 
Mean Squared Error: 0.0118 ± 0.3183
Root Mean Squared Error: 0.1085
Mean Absolute Error: -0.0748
R-squared: 0.7776 ± 0.3183
Relative Percent Difference: 0.4009

Testing Set: 
Mean Squared Error: 0.0140 ± 0.5063
Root Mean Squared Error: 0.1181
Mean Absolute Error: -0.0961
R-squared: 0.5196 ± 0.5063
Relative Percent Difference: 0.3935


In [80]:
### 2. Decision Tree

In [81]:
model2 = DecisionTreeRegressor()

In [82]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

Y_pred_train = cross_val_predict(model2, X_train, Y_train, cv=cv)

mse_train = -np.mean(cross_val_score(model2, X_train, Y_train, cv=cv, scoring='neg_mean_squared_error'))
rmse_train = np.sqrt(mse_train)
mae_train = np.mean(cross_val_score(model2, X_train, Y_train, cv=cv, scoring='neg_mean_absolute_error'))
r2_train_scores = cross_val_score(model2, X_train, Y_train, cv=cv, scoring='r2')
r2_train_mean = np.mean(r2_train_scores)

residuals_train = Y_train - Y_pred_train
std_dev_res_train = np.std(residuals_train)
mean_y_train = np.mean(Y_train)
rpd_train = std_dev_res_train / mean_y_train

print('\nTraining Set: ')
print(f'Mean Squared Error: {mse_train}')
print(f'Root Mean Squared Error: {rmse_train}')
print(f'Mean Absolute Error: {mae_train}')
print(f'R-squared: {r2_train_mean}')
print(f'Relative Percent Difference: {rpd_train}')

Y_pred_test = cross_val_predict(model2, X_test, Y_test, cv=cv)

mse_test = -np.mean(cross_val_score(model2, X_test, Y_test, cv=cv, scoring='neg_mean_squared_error'))
rmse_test = np.sqrt(mse_test)
mae_test = np.mean(cross_val_score(model2, X_test, Y_test, cv=cv, scoring='neg_mean_absolute_error'))
r2_test_scores = cross_val_score(model2, X_test, Y_test, cv=cv, scoring='r2')
r2_test_mean = np.mean(r2_test_scores)

residuals_test = Y_test - Y_pred_test
std_dev_res_test = np.std(residuals_test)
mean_y_test = np.mean(Y_test)
rpd_test = std_dev_res_test / mean_y_test

print('\nTesting Set: ')
print(f'Mean Squared Error: {mse_test}')
print(f'Root Mean Squared Error: {rmse_test}')
print(f'Mean Absolute Error: {mae_test}')
print(f'R-squared: {r2_test_mean}')
print(f'Relative Percent Difference: {rpd_test}')


Training Set: 
Mean Squared Error: 0.01300945634920635
Root Mean Squared Error: 0.11405900380595278
Mean Absolute Error: -0.07708333333333334
R-squared: 0.7976555152127707
Relative Percent Difference: 0.433406889527975

Testing Set: 
Mean Squared Error: 0.01658416666666667
Root Mean Squared Error: 0.1287795273584535
Mean Absolute Error: -0.08316666666666668
R-squared: 0.4525244575914648
Relative Percent Difference: 0.4287094218344937


In [83]:
### 3. SVR

In [84]:
model3 = SVR()

In [85]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

Y_pred_train = cross_val_predict(model3, X_train, Y_train, cv=cv)

mse_train = -np.mean(cross_val_score(model3, X_train, Y_train, cv=cv, scoring='neg_mean_squared_error'))
rmse_train = np.sqrt(mse_train)
mae_train = np.mean(cross_val_score(model3, X_train, Y_train, cv=cv, scoring='neg_mean_absolute_error'))
r2_train_scores = cross_val_score(model3, X_train, Y_train, cv=cv, scoring='r2')
r2_train_mean = np.mean(r2_train_scores)

residuals_train = Y_train - Y_pred_train
std_dev_res_train = np.std(residuals_train)
mean_y_train = np.mean(Y_train)
rpd_train = std_dev_res_train / mean_y_train

print('\nTraining Set: ')
print(f'Mean Squared Error: {mse_train}')
print(f'Root Mean Squared Error: {rmse_train}')
print(f'Mean Absolute Error: {mae_train}')
print(f'R-squared: {r2_train_mean}')
print(f'Relative Percent Difference: {rpd_train}')

Y_pred_test = cross_val_predict(model3, X_test, Y_test, cv=cv)

mse_test = -np.mean(cross_val_score(model3, X_test, Y_test, cv=cv, scoring='neg_mean_squared_error'))
rmse_test = np.sqrt(mse_test)
mae_test = np.mean(cross_val_score(model3, X_test, Y_test, cv=cv, scoring='neg_mean_absolute_error'))
r2_test_scores = cross_val_score(model3, X_test, Y_test, cv=cv, scoring='r2')
r2_test_mean = np.mean(r2_test_scores)

residuals_test = Y_test - Y_pred_test
std_dev_res_test = np.std(residuals_test)
mean_y_test = np.mean(Y_test)
rpd_test = std_dev_res_test / mean_y_test

print('\nTesting Set: ')
print(f'Mean Squared Error: {mse_test}')
print(f'Root Mean Squared Error: {rmse_test}')
print(f'Mean Absolute Error: {mae_test}')
print(f'R-squared: {r2_test_mean}')
print(f'Relative Percent Difference: {rpd_test}')


Training Set: 
Mean Squared Error: 0.013584567983037543
Root Mean Squared Error: 0.11655285489012074
Mean Absolute Error: -0.08490116199931508
R-squared: 0.8050480817751653
Relative Percent Difference: 0.4253545881044649

Testing Set: 
Mean Squared Error: 0.01529171568244099
Root Mean Squared Error: 0.12365967686534277
Mean Absolute Error: -0.08863499373748177
R-squared: 0.7025287822255241
Relative Percent Difference: 0.3946795336525856


In [86]:
### 4. ANN

In [87]:
os.environ['PYTHONHASHSEED'] = '42'
tf.random.set_seed(42)
#tf.config.threading.set_inter_op_parallelism_threads(1)
#tf.config.threading.set_intra_op_parallelism_threads(1)
np.random.seed(42)

In [88]:
def create_model():
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [89]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

mse_train_list = []
rmse_train_list = []
mae_train_list = []
r2_train_list = []

mse_test_list = []
rmse_test_list = []
mae_test_list = []
r2_test_list = []

for train_index, test_index in cv.split(X_train):
    X_cv_train, X_cv_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_cv_train, y_cv_test = Y_train.iloc[train_index], Y_train.iloc[test_index]

    model = create_model()

    # Set up early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    model.fit(X_cv_train, y_cv_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=0)

    # Predict on the training set
    y_pred_train = model.predict(X_cv_train).flatten()

    # Calculate training set metrics
    mse_train = mean_squared_error(y_cv_train, y_pred_train)
    rmse_train = np.sqrt(mse_train)
    mae_train = mean_absolute_error(y_cv_train, y_pred_train)
    r2_train = 1 - mse_train / np.var(y_cv_train)

    mse_train_list.append(mse_train)
    rmse_train_list.append(rmse_train)
    mae_train_list.append(mae_train)
    r2_train_list.append(r2_train)

    # Predict on the test set
    y_pred_test = model.predict(X_cv_test).flatten()

    # Calculate testing set metrics
    mse_test = mean_squared_error(y_cv_test, y_pred_test)
    rmse_test = np.sqrt(mse_test)
    mae_test = mean_absolute_error(y_cv_test, y_pred_test)
    r2_test = 1 - mse_test / np.var(y_cv_test)

    mse_test_list.append(mse_test)
    rmse_test_list.append(rmse_test)
    mae_test_list.append(mae_test)
    r2_test_list.append(r2_test)

# Calculate mean metrics over all folds for training set
mean_mse_train = np.mean(mse_train_list)
mean_rmse_train = np.mean(rmse_train_list)
mean_mae_train = np.mean(mae_train_list)
mean_r2_train = np.mean(r2_train_list)

# Calculate mean metrics over all folds for testing set
mean_mse_test = np.mean(mse_test_list)
mean_rmse_test = np.mean(rmse_test_list)
mean_mae_test = np.mean(mae_test_list)
mean_r2_test = np.mean(r2_test_list)

# Print the results for training set
print('Training Set: ')
print(f'Mean Squared Error: {mean_mse_train}')
print(f'Root Mean Squared Error: {mean_rmse_train}')
print(f'Mean Absolute Error: {mean_mae_train}')
print(f'R-squared: {mean_r2_train}')

# Print the results for testing set
print('\nTesting Set: ')
print(f'Mean Squared Error: {mean_mse_test}')
print(f'Root Mean Squared Error: {mean_rmse_test}')
print(f'Mean Absolute Error: {mean_mae_test}')
print(f'R-squared: {mean_r2_test}')

1/1 [==============================] - 0s 90ms/step
Training Set: 
Mean Squared Error: 0.013467379301599244
Root Mean Squared Error: 0.11597807708444965
Mean Absolute Error: 0.08090046562202899
R-squared: 0.8489878863062004

Testing Set: 
Mean Squared Error: 0.014943381452810166
Root Mean Squared Error: 0.12002454135717786
Mean Absolute Error: 0.08468260684212048
R-squared: 0.7910787494709415
